In [ ]:
import pandas as pd
import numpy as np
import json

# Load the training data
train_path = '/home/data/train.json'

# Try reading the JSON file as a single JSON object
with open(train_path, 'r') as f:
    data = json.load(f)

print(f"Type of data: {type(data)}")
if isinstance(data, list):
    train_df = pd.DataFrame(data)
    print(f"Training data shape: {train_df.shape}")
    print(f"Columns: {list(train_df.columns)}")
else:
    print(f"Data structure: {data.keys() if isinstance(data, dict) else 'Not a dict'}")